# Create new test and validation sets

In [ ]:
import pandas as pd
import s3fs
import boto3
import json
from datetime import datetime
import numpy as np

In [14]:
SAGEMAKER_REPO_PATH = r'/home/ec2-user/SageMaker/classify-streetview'

In [2]:
df_results = pd.read_csv('all_test_crops_model0.csv')
df_results.shape

(3448, 8)

In [3]:
df_results = df_results.rename(columns = {'Unnamed: 0.1' : 'image_name'})
df_results.head()


,Unnamed: 0,image_name,prediction,0,1,2,3,4
0,0,8475_135_1.jpg,1_null,-1.133551,4.186599,-1.984925,1.529789,-2.481364
1,1,8475_135_2.jpg,1_null,-0.763638,3.575558,-1.084529,1.076634,-2.774253
2,2,8475_135_3.jpg,1_null,-1.193419,4.510900,-2.012558,0.620944,-1.824119
3,3,8475_135_4.jpg,1_null,-0.106576,3.914730,-2.002850,1.609854,-3.344971
4,4,8475_135_5.jpg,1_null,-1.827574,4.439346,-1.939240,0.628309,-1.315913


In [4]:
df_results.columns

Index(['Unnamed: 0', 'image_name', 'prediction', '0', '1', '2', '3', '4'], dtype='object')

In [5]:
df_results = df_results[['image_name', 'prediction', '0', '1', '2', '3', '4']]

In [7]:
df_test = pd.read_csv(os.path.join(SAGEMAKER_REPO_PATH, 'label-crops/test_crop_filename.csv'))
df_test.head()

,Unnamed: 0,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all,img_heading_folder,original_image_path,new_image_path,image_name
0,0,141,45,1,0,1,0,0,0,0,0,0,141_45,crops/141_45/1.jpg,labeled-test/141_45_1.jpg,141_45_1.jpg
1,1,141,45,2,0,1,0,0,0,0,0,0,141_45,crops/141_45/2.jpg,labeled-test/141_45_2.jpg,141_45_2.jpg
2,2,141,45,3,0,1,0,0,0,0,0,0,141_45,crops/141_45/3.jpg,labeled-test/141_45_3.jpg,141_45_3.jpg
3,3,141,45,4,0,1,0,0,0,0,0,0,141_45,crops/141_45/4.jpg,labeled-test/141_45_4.jpg,141_45_4.jpg
4,4,141,45,5,0,1,0,0,0,0,0,0,141_45,crops/141_45/5.jpg,labeled-test/141_45_5.jpg,141_45_5.jpg


In [8]:
df_test = df_test[['img_id', 'heading', 'crop_number', '0_missing', '1_null',
       '2_obstacle', '3_present', '4_surface_prob', '5_nosidewalk',
       '6_occlusion', 'count_all', 'image_name']]

In [9]:
df_combine = df_results.merge(df_test, how = 'left', left_on = 'image_name', right_on = 'image_name')
df_combine.head()

,image_name,prediction,0,1,2,3,4,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all
0,8475_135_1.jpg,1_null,-1.133551,4.186599,-1.984925,1.529789,-2.481364,8475,135,1,0,1,0,0,0,0,0,0
1,8475_135_2.jpg,1_null,-0.763638,3.575558,-1.084529,1.076634,-2.774253,8475,135,2,0,0,1,0,0,0,0,1
2,8475_135_3.jpg,1_null,-1.193419,4.510900,-2.012558,0.620944,-1.824119,8475,135,3,0,1,0,0,0,0,0,0
3,8475_135_4.jpg,1_null,-0.106576,3.914730,-2.002850,1.609854,-3.344971,8475,135,4,0,1,0,0,0,0,0,0
4,8475_135_5.jpg,1_null,-1.827574,4.439346,-1.939240,0.628309,-1.315913,8475,135,5,0,0,1,0,0,0,0,1


In [10]:
df_combine['ground_truth'] = '1_null'
df_combine['ground_truth'] = np.where(df_combine['3_present'] > 0, '3_present', df_combine['ground_truth'])
df_combine['ground_truth'] = np.where(df_combine['0_missing'] > 0, '0_missing', df_combine['ground_truth'])
df_combine['ground_truth'] = np.where(df_combine['4_surface_prob'] > 0, '4_surface_prob', df_combine['ground_truth'])
df_combine['ground_truth'] = np.where(df_combine['2_obstacle'] > 0, '2_obstacle', df_combine['ground_truth'])
df_combine['ground_truth'].value_counts()

1_null            2268
3_present          718
4_surface_prob     250
2_obstacle         129
0_missing           83
Name: ground_truth, dtype: int64

In [11]:
df_combine['prediction'].value_counts()

1_null        3292
3_present      149
2_obstacle       7
Name: prediction, dtype: int64

In [12]:

df_combine.head()

,image_name,prediction,0,1,2,3,4,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all,ground_truth
0,8475_135_1.jpg,1_null,-1.133551,4.186599,-1.984925,1.529789,-2.481364,8475,135,1,0,1,0,0,0,0,0,0,1_null
1,8475_135_2.jpg,1_null,-0.763638,3.575558,-1.084529,1.076634,-2.774253,8475,135,2,0,0,1,0,0,0,0,1,2_obstacle
2,8475_135_3.jpg,1_null,-1.193419,4.510900,-2.012558,0.620944,-1.824119,8475,135,3,0,1,0,0,0,0,0,0,1_null
3,8475_135_4.jpg,1_null,-0.106576,3.914730,-2.002850,1.609854,-3.344971,8475,135,4,0,1,0,0,0,0,0,0,1_null
4,8475_135_5.jpg,1_null,-1.827574,4.439346,-1.939240,0.628309,-1.315913,8475,135,5,0,0,1,0,0,0,0,1,2_obstacle


In [13]:
df_combine.to_csv('model0predict_with_groundtruth.csv')

# Create new Test and Validation Sets

In [18]:
df_all = df_combine[['image_name', 'img_id', 'ground_truth']]
df_all['ground_truth'].value_counts()

1_null            2268
3_present          718
4_surface_prob     250
2_obstacle         129
0_missing           83
Name: ground_truth, dtype: int64

In [30]:
df_all['ground_truth'].value_counts() / 2

1_null            1134.0
3_present          359.0
4_surface_prob     125.0
2_obstacle          64.5
0_missing           41.5
Name: ground_truth, dtype: float64

In [25]:
df_all.shape

(3448, 3)

In [31]:
3448 / 2

1724.0

In [26]:
imgid_list = list(df_all['img_id'].unique())
print(len(imgid_list))
imgid_list[0:5]

124


[8475, 8503, 8540, 8650, 8780]

In [35]:
valid_imgid = random.sample(imgid_list, 62)
len(valid_imgid)
df_valid = df_all.loc[df_all['img_id'].isin(valid_imgid)]
print(df_valid.shape)
df_valid['ground_truth'].value_counts()

(1712, 3)


1_null            1139
3_present          354
4_surface_prob     117
2_obstacle          65
0_missing           37
Name: ground_truth, dtype: int64

In [36]:
df_valid.head()

,image_name,img_id,ground_truth
28,8503_135_1.jpg,8503,1_null
29,8503_135_2.jpg,8503,1_null
30,8503_135_3.jpg,8503,1_null
31,8503_135_4.jpg,8503,1_null
32,8503_135_5.jpg,8503,1_null


In [37]:
df_valid.to_csv('validation_set_from_test.csv', index = False)

In [38]:
df_all['is_valid_set'] = np.where(df_all['img_id'].isin(valid_imgid), True, False)
df_all.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,image_name,img_id,ground_truth,is_valid_set
0,8475_135_1.jpg,8475,1_null,False
1,8475_135_2.jpg,8475,2_obstacle,False
2,8475_135_3.jpg,8475,1_null,False
3,8475_135_4.jpg,8475,1_null,False
4,8475_135_5.jpg,8475,2_obstacle,False


In [39]:
df_all['is_valid_set'].value_counts()

False    1736
True     1712
Name: is_valid_set, dtype: int64

In [41]:
df_all.to_csv('validation_test_includeallnull.csv', index = False)

In [40]:
df_new_test = df_all.loc[~df_all['is_valid_set']]
df_new_test['ground_truth'].value_counts()

1_null            1129
3_present          364
4_surface_prob     133
2_obstacle          64
0_missing           46
Name: ground_truth, dtype: int64

# Remove Nulls

In [58]:
df_null = df_all.loc[df_all['ground_truth'].str.contains('1_null')]
df_to_remove = df_null.sample(n = 1548) #Keep 720 total
remove_imgid_list = list(df_to_remove['image_name'])
df_new_all = df_all.loc[~df_all['image_name'].isin(remove_imgid_list)]
#df_new_all['ground_truth'].value_counts()
df_new_all.groupby(['ground_truth', 'is_valid_set'])['image_name', 'img_id'].nunique()

image_name  img_id
ground_truth   is_valid_set                    
0_missing      False                 46      16
               True                  37      13
1_null         False                364      62
               True                 356      62
2_obstacle     False                 64      17
               True                  65      12
3_present      False                364      54
               True                 354      56
4_surface_prob False                133      26
               True                 117      24

In [59]:
df_new_all.to_csv('validation_test_final.csv', index = False)

In [42]:
df_all['ground_truth'].value_counts()

1_null            2268
3_present          718
4_surface_prob     250
2_obstacle         129
0_missing           83
Name: ground_truth, dtype: int64

In [46]:
df_all.groupby(['ground_truth', 'is_valid_set'])['image_name', 'img_id'].nunique()

image_name  img_id
ground_truth   is_valid_set                    
0_missing      False                 46      16
               True                  37      13
1_null         False               1129      62
               True                1139      62
2_obstacle     False                 64      17
               True                  65      12
3_present      False                364      54
               True                 354      56
4_surface_prob False                133      26
               True                 117      24